### 환경 설정

In [6]:
import os
import json

import httpx
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
def write_json(data, filepath):
    with open(filepath, 'w') as file:
        json.dump(data, file, ensure_ascii=False)

In [8]:
url_collection = "https://api.reservoir.tools/collections/v7"
url_tokens = "https://api.reservoir.tools/tokens/v7"
url_transaction = "https://api.reservoir.tools/collections/activity/v6"
url_attributes = "https://api.reservoir.tools/collections/{collection}/attributes/all/v4"


headers = {
    "x-api-key": os.getenv("RESERVOIR_API_KEY"),
    "accept": "*/*"
}

In [9]:
path_output = "./2nd"
path_tokens = os.path.join(path_output, "tokens")
path_transactions = os.path.join(path_output, "transactions")
path_collections = os.path.join(path_output, "collections")
path_attributes = os.path.join(path_output, "attributes")

output_collection_name = "{network}_{collection_id}_collection.json"
output_tokens_name = "{network}_{collection_id}_{idx:06d}_tokens.json"
output_transactions_name = "{network}_{collection_id}_{idx:06d}_transactions.json"
output_attributes_name = "{network}_{collection_id}_attributes.json"

os.makedirs(path_output, exist_ok=True)
os.makedirs(path_tokens, exist_ok=True)
os.makedirs(path_transactions, exist_ok=True)
os.makedirs(path_collections, exist_ok=True)
os.makedirs(path_attributes, exist_ok=True)

In [10]:
network = "ethereum"
collection_id = "0xed5af388653567af2f388e6224dc7c4b3241c544"

### Collections

In [ ]:
collection = httpx.get(url_collection, params={"id": collection_id}, headers=headers).json()
write_json(collection, os.path.join(path_collections, output_collection_name.format(network=network, collection_id=collection_id)))
collection

### Tokens

In [ ]:
idx = 0
continuation = ""

while True:
    params = {
        "collection": collection_id,
        "limit": 1000,
        "sortBy": "updatedAt",
    }
    
    if continuation:
        params["continuation"] = continuation
        
    print(params)
    
    resp = httpx.get(url_tokens, params=params, headers=headers).json()
    write_json(resp, os.path.join(path_tokens, output_tokens_name.format(network=network, collection_id=collection_id, idx=idx)))
    
    print(f"idx: {idx}, Downloaded {len(resp['tokens'])} tokens, continuation: {resp['continuation']}")
    if len(resp["tokens"]) < 1000 or resp["continuation"] is None:
        break
    
    idx += 1
    continuation = resp["continuation"]

### Transactions

In [12]:
idx = 4540
continuation = "MTY1MjQ5NjY3Nl80MmZiODM4NzE5MWUxOWYxMjk4YTE1MGJmNDk2NDhiNTdlYzM0ZWYzNjM0ZDFiYjQxYzJiODM3ZDYwOWNmNjVk"

while True:
    params = {
        "collection": collection_id,
        "limit": 50,
        "types": ["sale", "transfer", "mint"]
    }
    
    if continuation:
        params["continuation"] = continuation
        
    print(params)
    
    resp = httpx.get(url_transaction, params=params, headers=headers, timeout=30).json()
    write_json(resp, os.path.join(path_transactions, output_transactions_name.format(network=network, collection_id=collection_id, idx=idx)))
    
    print(f"idx: {idx}, Downloaded {len(resp['activities'])} activities, continuation: {resp['continuation']}")
    if len(resp["activities"]) < 50 or resp["continuation"] is None:
        break
    
    idx += 1
    continuation = resp["continuation"]

{'collection': '0xed5af388653567af2f388e6224dc7c4b3241c544', 'limit': 50, 'types': ['sale', 'transfer', 'mint'], 'continuation': 'MTY1MjQ5NjY3Nl80MmZiODM4NzE5MWUxOWYxMjk4YTE1MGJmNDk2NDhiNTdlYzM0ZWYzNjM0ZDFiYjQxYzJiODM3ZDYwOWNmNjVk'}
idx: 4540, Downloaded 50 activities, continuation: MTY1MjQ3OTk3Nl80MmE5NTg1ZDU1YzVlMWU4ZjU2YzQ2N2Y0NjU0NmFlOWJjMzc3YWViMjI5NDNiOWZjMzg4YzRkYjVmZjA0YzRi
{'collection': '0xed5af388653567af2f388e6224dc7c4b3241c544', 'limit': 50, 'types': ['sale', 'transfer', 'mint'], 'continuation': 'MTY1MjQ3OTk3Nl80MmE5NTg1ZDU1YzVlMWU4ZjU2YzQ2N2Y0NjU0NmFlOWJjMzc3YWViMjI5NDNiOWZjMzg4YzRkYjVmZjA0YzRi'}
idx: 4541, Downloaded 50 activities, continuation: MTY1MjQ3MTc4N19kZDg4MWZiYjkyYTQ2N2E1N2U5YTNhZTJlMDM1ZGY5Yzc0YjhhZjY4Zjc1YjdmNWUwYTdkNTlmYmQzYWFlMDE1
{'collection': '0xed5af388653567af2f388e6224dc7c4b3241c544', 'limit': 50, 'types': ['sale', 'transfer', 'mint'], 'continuation': 'MTY1MjQ3MTc4N19kZDg4MWZiYjkyYTQ2N2E1N2U5YTNhZTJlMDM1ZGY5Yzc0YjhhZjY4Zjc1YjdmNWUwYTdkNTlmYmQzYWFlMDE1

### Attributes

In [13]:
resp = httpx.get(url_attributes.format(collection=collection_id), headers=headers).json()
write_json(resp, os.path.join(path_attributes, output_attributes_name.format(network=network, collection_id=collection_id)))
resp

{'attributes': [{'key': 'Special',
   'attributeCount': 9,
   'kind': 'string',
   'values': [{'count': 86, 'value': 'Fox Fire'},
    {'count': 48,
     'value': 'Lightning',
     'floorAskPrice': {'currency': {'contract': '0x0000000000000000000000000000000000000000',
       'name': 'Ether',
       'symbol': 'ETH',
       'decimals': 18},
      'amount': {'raw': '28000000000000000000',
       'decimal': 28,
       'usd': 82552.49674,
       'native': 28}}},
    {'count': 80,
     'value': 'Sakura',
     'floorAskPrice': {'currency': {'contract': '0x0000000000000000000000000000000000000000',
       'name': 'Ether',
       'symbol': 'ETH',
       'decimals': 18},
      'amount': {'raw': '9149999900000000000',
       'decimal': 9.15,
       'usd': 26976.97632,
       'native': 9.15}}},
    {'count': 81,
     'value': 'Smoke',
     'floorAskPrice': {'currency': {'contract': '0x0000000000000000000000000000000000000000',
       'name': 'Ether',
       'symbol': 'ETH',
       'decimals': 18},